In [1]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11,8)})
sns.set(font_scale=1.2)
import numpy as np
from utils import fix_wrong_record_id, fill_id, format_datetime_columns, delete_redundant_columns, mark_inattentive_participants, fix_age,remove_test_id, delete_negative_age, fill_missing_age, fill_age
from merge_datasets import split_to_intersection_and_union, fix_wrong_groups, resolve_contradiction
from columns_index_for_merging import columns_dict, merging_functions_by_columns_set, weird_text_columns

In [2]:
columns_dict['duplicated_cols']

['redcap_survey_identifier',
 'er_num',
 'complaint_other',
 'diagnosis_other',
 'opening_psychothe_3',
 'opening_medicine_3',
 'iat_done',
 'consent',
 'consent_no',
 'covid_1___1',
 'covid_1___2',
 'covid_1___3',
 'covid_1___4',
 'covid_2',
 'covid_3___1',
 'covid_3___2',
 'covid_3___3',
 'covid_3___4',
 'covid_3___5',
 'covid_3___6',
 'covid_3___7',
 'covid_3___8',
 'covid_4',
 'covid_5',
 'covid_6',
 'covid_7',
 'covid_8',
 'gender',
 'age_child',
 'maris_sci_sf_1',
 'maris_sci_sf_2',
 'maris_sci_sf_3',
 'maris_sci_sf_4',
 'maris_sci_sf_5',
 'maris_sci_sf_6',
 'maris_sci_sf_7',
 'maris_sci_sf_8',
 'maris_sci_sf_9',
 'maris_soq_sf_1',
 'maris_soq_sf_2',
 'maris_soq_sf_3',
 'maris_soq_sf_4',
 'maris_soq_sf_5',
 'maris_soq_sf_6',
 'maris_soq_sf_7',
 'maris_soq_sf_8',
 'c_ssrs_1_life',
 'c_ssrs_2_life',
 'c_ssrs_3_life',
 'c_ssrs_4_life',
 'c_ssrs_5_life',
 'c_ssrs_6_life',
 'c_ssrs_1_2weeks',
 'c_ssrs_2_2weeks',
 'c_ssrs_3_2weeks',
 'c_ssrs_4_2weeks',
 'c_ssrs_5_2weeks',
 'c_ssrs_6_2w

In [2]:
def impute_from_column(df, impute_to, impute_from):
    """
    test:
    df[(df['c_ssrs_6'].isna()) & (~df['c_ssrs_last_visit_6'].isna())][['c_ssrs_6', 'c_ssrs_last_visit_6']]
    df = impute_from_column(df, impute_to = 'c_ssrs_6', impute_from = 'c_ssrs_last_visit_6')
    df[(df['c_ssrs_6'].isna()) & (~df['c_ssrs_last_visit_6'].isna())][['c_ssrs_6', 'c_ssrs_last_visit_6']]
    
    """
    df[impute_to] = np.where(df[impute_to].isnull(), df[impute_from], df[impute_to])
    
    return df

In [3]:
df_2022 = pd.read_csv(r'../preprocessed_data/2022_data_imputed_with_clin_stu.csv', na_values= ' ')
df_2021 = pd.read_csv(r'../preprocessed_data/2021_data_imputed_with_clin_stu.csv', na_values= ' ')

df_2022 = delete_negative_age(df_2022)
df_2021 = delete_negative_age(df_2021)

df_2021 = impute_from_column(df_2021, impute_to = 'age_child_pre', impute_from = 'age_child_pre_first')

C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\203203614.py:1: DtypeWarning: Columns (44,46,51,53,99,325,366,439,453,459,464,466,501,507,512,514,519,521,530,532,569,571,580,599,766,771,773,779,784,789,794,798,800,804,809,811,813,816,818,827,829,849,866,868,877,896,906,941,964,1002,1014,1025,1063,1068,1070,1076,1081,1086,1091,1093,1095,1097,1099,1100,1126,1142,1149,1165,1168,1172,1173,1177,1178,1179,1180,1182,1183,1184,1187,1188,1190,1207,1212,1229,1232,1260,1262,1264,1266,1268,1271,1281,1283,1285,1291,1293,1297,1305,1306,1319,1321,1324,1326,1338,1340,1342,1343,1344,1350,1351,1352,1353,1465,1467,1468,1470,1486,1489,1492,1496,1513,1515,1516,1517,1523,1524,1525,1526,1580,1650,1653,1654) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv(r'../preprocessed_data/2022_data_imputed_with_clin_stu.csv', na_values= ' ')
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\203203614.py:2: DtypeWarning: Columns (48,1109,1166,1171,1344,1347,1563,1564,15

# the preparing datasets

In [4]:
df_2021 = fix_wrong_record_id(df_2021, year=2021)
df_2022 = fix_wrong_record_id(df_2022, year=2022)

##### fill id function - only for 2022

In [5]:
df_2022 = fill_id(df_2022)

In [6]:
df_2021 = format_datetime_columns(df_2021)
df_2022 = format_datetime_columns(df_2022)

C:\Users\nogur\Documents\DeppClinic\creating_the_clinic_dataset\step2_merge_2021_2022\utils.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\nogur\Documents\DeppClinic\creating_the_clinic_dataset\step2_merge_2021_2022\utils.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\nogur\Documents\DeppClinic\creating_the_clinic_dataset\step2_merge_2021_2022\utils.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors

In [7]:
df_2021 = mark_inattentive_participants(df_2021, year=2021)
df_2022 = mark_inattentive_participants(df_2022, year=2022)

In [8]:
df_2021, df_2022 = delete_redundant_columns(df_2021 = df_2021, df_2022 = df_2022)

# merging the datasets

In [9]:
union, intersection_2021, intersection_2022 = split_to_intersection_and_union(df_2021 = df_2021, df_2022=df_2022)

In [10]:
union.shape

(2377, 1625)

## handling contradictions

In [11]:
intersection_df = pd.merge(intersection_2021, intersection_2022, on=['id', 'redcap_event_name'], how='outer', suffixes=('_2021', '_2022'))

In [12]:
intersection_df.shape

(404, 3131)

In [13]:
# create intersection_columns_list.csv file
#pd.DataFrame(intersection_2021.columns).to_csv('intersection_columns_list.csv', index=False)

In [14]:
result = intersection_df.copy()

for key, value in merging_functions_by_columns_set.items():
    print(key, value)
    columns = columns_dict[key]
    for col in columns:
        result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
        result = result.drop([f'{col}_2021', f'{col}_2022'], axis=1)

## iterate over the other columns
        
# validate
# i = columns.iloc[0]
# result2[(~result2[f'{i}_2022'].isna()) & (~result2[f'{i}_2021'].isna())][[i, f'{i}_2021', f'{i}_2022']]

visit_date_stu <built-in function max>
diagnosis_binary <built-in function max>
complaint_binary <built-in function max>
timestamp <built-in function min>


C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)


date <built-in function min>


C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: Dat

groups <built-in function max>


...

C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: Dat

C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: Dat

C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: Dat

C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: Dat

C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
C:\Users\nogur\AppData\Local\Temp\ipykernel_18096\4092734466.py:7: PerformanceWarning: Dat

In [15]:
intersection = fix_wrong_groups(result)
intersection['original_dataset'] = 'both 2021 & 2022'

In [16]:
'done'

'done'

In [17]:
# dd = pd.DataFrame(intersection.columns)
# dd[dd[0].duplicated()].to_csv('duplicated_cols.csv')

In [18]:
merged_df = pd.concat([union, intersection])

In [19]:
merged_df = remove_test_id(merged_df)
merged_df = fill_missing_age(merged_df)


#do it for gender too
merged_df = fill_age(merged_df)

In [20]:
#[i for i in merged_df.columns if 'age' in i.lower()]


In [21]:
merged_df.query("age_child_pre < 0")[["age_child", "age_child_pre", "id"]]

,age_child,age_child_pre,id


In [22]:
(merged_df.groupby('id').age_child.count() == 0).sum()

523

In [23]:
a = (merged_df.groupby('id').age_child_pre.count() == 0)

In [24]:
a[a].sum()

38

In [25]:
a[a].to_csv("missing_age_part_2.csv")

In [26]:
merged_df.to_csv(r'../preprocessed_data/merged_2021_and_2022.csv', index=False)